In [3]:
#pip install transformers


In [1]:
pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 17.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [2]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4

In [4]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

Importing the model

In [5]:


model_name = "google/flan-t5-small"
# Check if CUDA is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
foundation_model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device) # Move the model to the device


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Checking my model how it performs in zero shot for summarization

In [6]:
# This is just to check the model with no fine tuning



resume_text = "John Doe | Software Developer Experience: Java, Python, and C++ (4 years) Agile methodologies, RESTful APIs, and database management Education: Bachelor's degree in Computer Science, XYZ University"

input1 = tokenizer(f"Resume: {resume_text}\nSummarize this Resume:", return_tensors="pt").to(device)

foundation_outputs = foundation_model.generate(
    input_ids=input1["input_ids"],
    attention_mask=input1["attention_mask"],
    max_new_tokens=150,
    do_sample=True,
    temperature=0.3,
    eos_token_id=tokenizer.eos_token_id,
    top_k=50,  # Use top-k sampling
    top_p=0.9  # Use top-p (nucleus) sampling
    )
print(tokenizer.batch_decode(foundation_outputs, skip_special_tokens=True))


['John Doe is a software developer who is a software engineer. He is a software engineer who is a software engineer.']


In [7]:
from google.colab import drive
import os
# Mount Google Drive
drive.mount('/content/drive')

import os

output_directory = "/content/drive/MyDrive/Colab Notebooks/Text Summarization/"

if not os.path.exists(output_directory):
    os.mkdir(output_directory)

path = '/content/drive/MyDrive/Colab Notebooks/Text Summarization'
files = os.listdir(path)
print(files)

Mounted at /content/drive
['Datasets', 'working_dir', 'runs', 'peft_model_1720299458.6191595']


# **Load The Resume dataset**

In [8]:
from datasets import load_dataset, DatasetDict,Dataset

ds = load_dataset("burberg92/resume_summary",cache_dir="/content/drive/MyDrive/Colab Notebooks/Text Summarization/Datasets")
print(ds)


DatasetDict({
    train: Dataset({
        features: ['resume', 'ex_summary'],
        num_rows: 100
    })
})


# **Creating custom prompt template**

In [9]:
def create_prompt(example):
    return {'prompt': example['resume'], 'summary': example['ex_summary']}



In [10]:
# Define the function to create prompt formats
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters
    :param sample: Sample dictionary
    """
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Summarize the below conversation."
    RESPONSE_KEY = "### Summary:"
    END_KEY = "### End"

    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['prompt']}" if sample["prompt"] else None
    response = f"{RESPONSE_KEY}\n{sample['summary']}"
    end = f"{END_KEY}"

    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

# Apply the function to create the prompt dataset
prompt_dataset = ds.map(create_prompt)
prompt_dataset = prompt_dataset.remove_columns(['resume', 'ex_summary'])
print(prompt_dataset)
print('--------')
# Apply the function to create the prompt formats
formatted_prompt_dataset = prompt_dataset.map(create_prompt_formats)

# Check the first few examples to ensure the prompts are formatted correctly
formatted_prompt_dataset['train']


DatasetDict({
    train: Dataset({
        features: ['prompt', 'summary'],
        num_rows: 100
    })
})
--------


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'summary', 'text'],
    num_rows: 100
})

Splitting the data into training and validation data

In [11]:
from sklearn.model_selection import train_test_split
k = formatted_prompt_dataset['train'].train_test_split(test_size=0.2)

data_slpit=DatasetDict({
    'train':k['train'],
    'val':k['test']
})
print(data_slpit)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'summary', 'text'],
        num_rows: 80
    })
    val: Dataset({
        features: ['prompt', 'summary', 'text'],
        num_rows: 20
    })
})


Max lenght of tokens that the model can hold

In [12]:
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


get_max_length(foundation_model)

Found max lenth: 512


512

Tokenization

Finding maximum token size

In [13]:
# Function to tokenize the prompts
def tokenize_and_find_length(example):
    tokens = tokenizer(example['text'], truncation=False)
    example['input_length'] = len(tokens['input_ids'])
    return example

# Apply the function to the dataset to find the length of each tokenized sequence
tokenized_lengths_dataset = formatted_prompt_dataset.map(tokenize_and_find_length, batched=False)

# Find the maximum token length in the dataset
max_token_length = max(tokenized_lengths_dataset['train']['input_length'])
print(f"Maximum token length in the dataset: {max_token_length}")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Maximum token length in the dataset: 286


In [14]:
def tokenize_function(example):
    model_inputs = tokenizer(example['text'], truncation=True, padding='max_length', max_length=max_token_length)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example['summary'], truncation=True, padding='max_length', max_length=max_token_length)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset = data_slpit.map(tokenize_function, batched=True, remove_columns=['prompt', 'summary', 'text'])
print(tokenized_dataset['train'][0])
print(tokenized_dataset['val'][0])

train_sample=tokenized_dataset['train']
val_sample=tokenized_dataset['val']


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

{'input_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7255, 19, 46, 8033, 24, 8788, 3, 9, 2491, 5, 8733, 3, 9, 1773, 24, 18056, 743, 7, 8, 1690, 5, 1713, 30345, 86, 7593, 10, 12198, 1635, 1737, 8, 666, 3634, 5, 24384, 3899, 1820, 9185, 21166, 9020, 24797, 11573, 10, 9185, 4193, 6, 799, 11, 387, 463, 4891, 6, 11, 8253, 5856, 16366, 585, 11, 18355, 1098, 16013, 7, 10, 9185, 15186, 27792, 41, 427, 2965, 61, 4751, 23023, 10, 9185, 2854, 3037, 29, 6, 283, 7400, 9185, 25195, 3, 31606, 28, 3262, 4193, 11, 4891, 4145, 22097, 15, 26, 1103, 16, 799, 11, 387, 463, 4891, 11, 8253, 5856, 9185, 21166, 6, 276, 2247, 448, 16268, 7, 41, 10218, 7988, 2773, 61, 1193, 21336, 3262, 16396, 11, 4891, 1087, 3, 18521, 26, 8253, 5856, 11, 937, 4864, 30, 3262, 200, 2869, 2855, 10, 10199, 31, 7, 1952, 16, 9185, 2854, 6, 180, 9968, 636, 

In [15]:
# # Function to detokenize the token IDs
# def detokenize_function(example):
#     example['decoded_text'] = tokenizer.decode(example['input_ids'], skip_special_tokens=True)
#     #example['decoded_summary'] = tokenizer.decode(example['labels'], skip_special_tokens=True)
#     return example

# # Apply the detokenization function to the validation sample
# detokenized_val_sample = val_sample.map(detokenize_function, batched=False)

# # Print the first few examples to see the results
# for i in range(1):
#     print(f"Decoded Text {i+1}: {detokenized_val_sample[i]['decoded_text']}")
#     #print(f"Decoded Summary {i+1}: {detokenized_val_sample[i]['decoded_summary']}\n")

# **Random Initialization**

In [30]:
from peft import  get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit

peft_config = PromptTuningConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    prompt_tuning_init=PromptTuningInit.RANDOM,
    num_virtual_tokens=6,
    tokenizer_name_or_path=model_name
)
peft_model = get_peft_model(foundation_model, peft_config)


In [31]:
print(peft_model.print_trainable_parameters())


trainable params: 6,144 || all params: 76,967,296 || trainable%: 0.0080
None


Creating a directory for training parameters


In [18]:


import os

output_directory = "/content/drive/MyDrive/Colab Notebooks/Text Summarization/working_dir"

if not os.path.exists(output_directory):
    os.mkdir(output_directory)
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

In [19]:

#pip install accelerate -U
import transformers
import accelerate

print(transformers.__version__)
print(accelerate.__version__)


4.41.2
0.32.1


In [29]:
#%%
from transformers import Seq2SeqTrainingArguments
# training_args = Seq2SeqTrainingArguments(
#     output_dir=output_directory,          # output directory
#     evaluation_strategy="epoch",     # evaluation strategy to use
#     per_device_train_batch_size=2,   # batch size for training
#     per_device_eval_batch_size=2,    # batch size for evaluation
#     weight_decay=0.01,               # strength of weight decay
#     save_total_limit=1,              # limit the total amount of checkpoints
#     num_train_epochs=5,              # total number of training epochs
#     learning_rate=5e-3,              # learning rate
#     predict_with_generate=True       # Whether to use generate to calculate generative metrics (ROUGE, BLEU)

# )


training_args = Seq2SeqTrainingArguments(
    output_dir=output_directory,          # output directory
    evaluation_strategy="epoch",     # evaluation strategy to use
    per_device_train_batch_size=2,   # batch size for training
    per_device_eval_batch_size=2,    # batch size for evaluation
    weight_decay=0.01,               # strength of weight decay
    save_total_limit=1,              # limit the total amount of checkpoints
    num_train_epochs=7,              # total number of training epochs
    learning_rate=5e-3,              # learning rate
    predict_with_generate=True,      # Whether to use generate to calculate generative metrics (ROUGE, BLEU)
    logging_strategy="steps",        # Log at each logging step
    logging_steps=10,                # Log every 10 steps
    report_to="none"                 # Avoid logging to external services
)


In [ ]:

# for text generation DataCollatorForLanguageModeling is used
# for text summarization as it is a sequence to sequence task
from transformers import Seq2SeqTrainer,DataCollatorForSeq2Seq

# Prepare data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=foundation_model)
# trainer = Seq2SeqTrainer(
#     model=peft_model,  # We pass in the PEFT version of the foundation model, bloomz-560M
#     args=training_args,
#     tokenizer=tokenizer,
#     train_dataset=train_sample,
#     eval_dataset=val_sample,
#     data_collator=data_collator
# )
# Debugging: Print shapes of a sample batch to ensure consistency
# def debug_collator(data_collator, dataset):
#     sample_batch = [dataset[i] for i in range(4)]
#     batch = data_collator(sample_batch)
#     print("Input IDs shape:", batch['input_ids'].shape)
#     print("Labels shape:", batch['labels'].shape)
#     print("Input IDs:", batch['input_ids'])
#     print("Labels:", batch['labels'])

trainer = Seq2SeqTrainer(
    model=peft_model,                      # the instantiated 🤗 Transformers model to be trained
    args=training_args,                    # training arguments
    train_dataset=train_sample, # training dataset
    eval_dataset=val_sample, # evaluation dataset
    tokenizer=tokenizer,                   # tokenizer
    data_collator=data_collator            # data collator
)

# debug_collator(data_collator, train_sample)
trainer.train()




Epoch,Training Loss,Validation Loss


In [22]:
import time

time_now = time.time()
peft_model_path = os.path.join(output_directory, f"peft_model_{time_now}")
trainer.model.save_pretrained(peft_model_path,force_download=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [26]:

from peft import PeftModel
import torch



#load the trained model
loaded_model = PeftModel.from_pretrained(foundation_model,
                                         peft_model_path,
                                         is_trainable=False,
                                         torch_dtype=torch.float16 # Ensure model weights are in float16
                                         #device_map={"":0}
                                         ) # Load the model on the first available GPU (index 0)


# Function to generate summary from a prompt
def summarize(prompt, max_length=300, num_beams=4,length_penalty=2.0):
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

    # Generate summary
    loaded_model_outputs = loaded_model.generate(
        input_ids=inputs["input_ids"],#.to('cuda'),  # Move input tensors to GPU
        attention_mask=inputs["attention_mask"],#.to('cuda'),  # Move attention mask tensors to GPU
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=True,
        #length_penalty=length_penalty,
        temperature=0.2,
        eos_token_id=tokenizer.eos_token_id,
        #max_new_tokens=100,
        top_k=50,  # Use top-k sampling
        top_p=0.95

    )

    # Decode the output
    summary = tokenizer.batch_decode(loaded_model_outputs, skip_special_tokens=True)
    return summary[0]

# Example prompt
prompt = """
Resume:
William Johnson | Project Manager Experience: Managing cross-functional teams, risk mitigation, and resource allocation (6 years) Certified PMP and Scrum Master Strong background in IT and software development projects Education: Master's degree in Business Administration, MNO University
Summary:
"""

# Generate and print the summary
summary = summarize(prompt)
print("Prompt:", prompt)
print("Summary:", summary)

Prompt: 
Resume:
William Johnson | Project Manager Experience: Managing cross-functional teams, risk mitigation, and resource allocation (6 years) Certified PMP and Scrum Master Strong background in IT and software development projects Education: Master's degree in Business Administration, MNO University
Summary:

Summary: As a Project Manager, William Johnson is a Senior Project Manager. He is a Senior Project Manager and is a Senior Project Manager.


# **Text Initialization**

In [ ]:

text_peft_config = PromptTuningConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    prompt_tuning_init_text="Generate Resume Summary", # this provides a starter for the model to start searching for the best embeddings
    num_virtual_tokens=7, # this doesn't have to match the length of the text above
    tokenizer_name_or_path=model_name
)


text_peft_model = get_peft_model(foundation_model, text_peft_config)
print(text_peft_model.print_trainable_parameters())


trainable params: 7,168 || all params: 76,968,320 || trainable%: 0.0093
None


In [ ]:

training_args = Seq2SeqTrainingArguments(
    output_dir=output_directory,          # output directory
    evaluation_strategy="epoch",     # evaluation strategy to use
    per_device_train_batch_size=2,   # batch size for training
    per_device_eval_batch_size=2,    # batch size for evaluation
    weight_decay=0.01,               # strength of weight decay
    save_total_limit=1,              # limit the total amount of checkpoints
    num_train_epochs=5,              # total number of training epochs
    learning_rate=5e-5,              # learning rate
    predict_with_generate=True,      # Whether to use generate to calculate generative metrics (ROUGE, BLEU)
    logging_strategy="steps",        # Log at each logging step
    logging_steps=10,                # Log every 10 steps
    report_to="none"                 # Avoid logging to external services
)
text_trainer = Trainer(
    model=text_peft_model,
    args=training_args,
    train_dataset=train_sample,
    eval_dataset=val_sample,
    tokenizer=tokenizer,
    data_collator=data_collator
)

text_trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,35.386800,38.590141
2,35.882100,38.588055
3,34.385300,38.586544
4,35.044000,38.585575
5,35.824500,38.585274


TrainOutput(global_step=200, training_loss=35.41623489379883, metrics={'train_runtime': 900.7288, 'train_samples_per_second': 0.444, 'train_steps_per_second': 0.222, 'total_flos': 41534909644800.0, 'train_loss': 35.41623489379883, 'epoch': 5.0})

In [ ]:
# Save the model
time_now = time.time()
text_peft_model_path = os.path.join(output_directory, f"text_peft_model_{time_now}")
text_trainer.model.save_pretrained(text_peft_model_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Load model
loaded_text_model = PeftModel.from_pretrained(foundation_model,
    text_peft_model_path,
    is_trainable=False)   #device_map={"":0})

#loaded_text_model.to('cuda') # Move the entire model to the GPU



In [ ]:
# Generate output
text_outputs = loaded_text_model.generate(
    input_ids=input1["input_ids"],#.to('cuda'), # Move input tensors to GPU
    attention_mask=input1["attention_mask"],#.to('cuda'), # Move attention mask tensors to GPU
    max_new_tokens=40,
    eos_token_id=tokenizer.eos_token_id
)

In [ ]:
print(tokenizer.batch_decode(text_outputs, skip_special_tokens=True))


['John Doe is a software developer. He is a software engineer.']


# Using QLORA for the quantization technique.
The quantization techniques helps mostly for decreasing the bit size.
This helps running the model with lower computaion power.